In [40]:
from __future__ import print_function, division
import pandas as pd
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

In [41]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## 1. 加载和查看数据

In [42]:
path='./'
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176 entries, 0 to 1175
Data columns (total 36 columns):
user_id                     1176 non-null int64
Age                         1176 non-null int64
Attrition                   1176 non-null object
BusinessTravel              1176 non-null object
DailyRate                   1176 non-null int64
Department                  1176 non-null object
DistanceFromHome            1176 non-null int64
Education                   1176 non-null int64
EducationField              1176 non-null object
EmployeeCount               1176 non-null int64
EmployeeNumber              1176 non-null int64
EnvironmentSatisfaction     1176 non-null int64
Gender                      1176 non-null object
HourlyRate                  1176 non-null int64
JobInvolvement              1176 non-null int64
JobLevel                    1176 non-null int64
JobRole                     1176 non-null object
JobSatisfaction             1176 non-null int64
MaritalStatus          

In [43]:
train.set_index('user_id',inplace=True)
train.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1374,58,No,Travel_Rarely,605,Sales,21,3,Life Sciences,1,1938,4,Female,72,3,4,Manager,4,Married,17875,11761,4,Y,Yes,13,3,3,80,1,29,2,2,1,0,0,0
1092,45,No,Travel_Rarely,950,Research & Development,28,3,Technical Degree,1,1546,4,Male,97,3,1,Research Scientist,4,Married,2132,4585,4,Y,No,20,4,4,80,1,8,3,3,5,4,0,3
768,40,No,Travel_Rarely,300,Sales,26,3,Marketing,1,1066,3,Male,74,3,2,Sales Executive,1,Married,8396,22217,1,Y,No,14,3,2,80,1,8,3,2,7,7,7,5
569,36,No,Non-Travel,1434,Sales,8,4,Life Sciences,1,789,1,Male,76,2,3,Sales Executive,1,Single,7587,14229,1,Y,No,15,3,2,80,0,10,1,3,10,7,0,9
911,25,Yes,Travel_Frequently,599,Sales,24,1,Life Sciences,1,1273,3,Male,73,1,1,Sales Representative,4,Single,1118,8040,1,Y,Yes,14,3,4,80,0,1,4,3,1,0,1,0


In [44]:
test.set_index('user_id',inplace=True)
test.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
442,36,Non-Travel,635,Sales,10,4,Medical,1,592,2,Male,32,3,3,Sales Executive,4,Single,9980,15318,1,Y,No,14,3,4,80,0,10,3,2,10,3,9,7
1091,33,Travel_Rarely,575,Research & Development,25,3,Life Sciences,1,1545,4,Male,44,2,2,Manufacturing Director,2,Single,4320,24152,1,Y,No,13,3,4,80,0,5,2,3,5,3,0,2
981,35,Travel_Frequently,662,Sales,18,4,Marketing,1,1380,4,Female,67,3,2,Sales Executive,3,Married,4614,23288,0,Y,Yes,18,3,3,80,1,5,0,2,4,2,3,2
785,40,Travel_Rarely,1492,Research & Development,20,4,Technical Degree,1,1092,1,Male,61,3,3,Healthcare Representative,4,Married,10322,26542,4,Y,No,20,4,4,80,1,14,6,3,11,10,11,1
1332,29,Travel_Frequently,459,Research & Development,24,2,Life Sciences,1,1868,4,Male,73,2,1,Research Scientist,4,Single,2439,14753,1,Y,Yes,24,4,2,80,0,1,3,2,1,0,1,0


## 2.数据预处理   

训练数据中，除了id和label，一共有34个特征。     

其中，连续值变量有：    
['Age',  'DailyRate',  'DistanceFromHome',  'EmployeeNumber', 'EnvironmentSatisfaction',  'HourlyRate', 'JobInvolvement', 'JobLevel',  'JobSatisfaction',  'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']

离散值变量有 ：       
['BusinessTravel','Department','Education', 'EducationField','Gender','JobRole','MaritalStatus', 'Over18', 'OverTime']   

其他有：   
[ 'EmployeeCount','Attrition', 'user_id']


#### 1. 对离散值变量进行one-hot编码


In [45]:
# 分离训练数据中的 特征 和 目标类别
X = train.drop('Attrition',axis=1)
y=train['Attrition'].map({"No": 0, "Yes": 1})
# 离散变量
discrete = ['BusinessTravel','Department','Education', 'EducationField','Gender','JobRole','MaritalStatus', 'Over18', 'OverTime'] 
#对离散值进行one-hot encoding
BusinessTravel_dummy=pd.get_dummies(X['BusinessTravel'], prefix= 'BusinessTravel')
Department_dummy=pd.get_dummies(X['Department'], prefix= 'Department')
Education_dummy=pd.get_dummies(X['Education'], prefix= 'Education')
EducationField_dummy=pd.get_dummies(X['EducationField'], prefix= 'EducationField')
Gender_dummy=pd.get_dummies(X['Gender'], prefix= 'Gender')
JobRole_dummy=pd.get_dummies(X['JobRole'], prefix= 'JobRole')
MaritalStatus_dummy=pd.get_dummies(X['MaritalStatus'], prefix= 'MaritalStatus')
Over18_dummy=pd.get_dummies(X['Over18'], prefix= 'Over18')
OverTime_dummy=pd.get_dummies(X['OverTime'], prefix= 'OverTime')
X.drop(discrete,axis=1,inplace =True)

X=pd.concat([X,BusinessTravel_dummy,Department_dummy,Education_dummy,EducationField_dummy,Gender_dummy,
             JobRole_dummy,MaritalStatus_dummy,Over18_dummy,OverTime_dummy],axis =1)
X.head()


,Age,DailyRate,DistanceFromHome,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,Education_1,Education_2,Education_3,Education_4,Education_5,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Female,Gender_Male,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1374,58,605,21,1,1938,4,72,3,4,4,17875,11761,4,13,3,3,80,1,29,2,2,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1
1092,45,950,28,1,1546,4,97,3,1,4,2132,4585,4,20,4,4,80,1,8,3,3,5,4,0,3,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,1,0
768,40,300,26,1,1066,3,74,3,2,1,8396,22217,1,14,3,2,80,1,8,3,2,7,7,7,5,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0
569,36,1434,8,1,789,1,76,2,3,1,7587,14229,1,15,3,2,80,0,10,1,3,10,7,0,9,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0
911,25,599,24,1,1273,3,73,1,1,4,1118,8040,1,14,3,4,80,0,1,4,3,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1


In [46]:
y.head()

user_id
1374    0
1092    0
768     0
569     0
911     1
Name: Attrition, dtype: int64

#### 2. 切分数据为训练集和测试集 ,并作标准化处理

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  StandardScaler
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,  random_state=33)
stdsc = StandardScaler()
#transform our training feature
X_train_std = stdsc.fit_transform(X_train)
#print(X_train_std[0])
X_test_std = stdsc.fit_transform(X_test)

D:\install\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\install\Anaconda\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
D:\install\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\install\Anaconda\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


## 3.构建模型


### 逻辑回归   

In [48]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression()
lg.fit(X_train_std, y_train)
print("训练集准确率: ", lg.score(X_train_std, y_train))
print("测试集准确率: ", lg.score(X_test_std, y_test))

训练集准确率:  0.8967193195625759
测试集准确率:  0.8725212464589235


D:\install\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### 决策树

In [49]:
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier(criterion="entropy",random_state=30,splitter="random")
model.fit(X_train_std,y_train)
print("训练集准确率:",model.score(X_train_std,y_train))
print("测试集准确率: ", model.score(X_test_std, y_test))

训练集准确率: 1.0
测试集准确率:  0.8158640226628895


### Xgboost

In [50]:
from xgboost import XGBClassifier
model = XGBClassifier(
        learning_rate=0.01,#学习率
        n_estimators=3000,#步长
        max_depth=4,#深度
        objective='binary:logistic',
        seed=27
    )
model.fit(X_train_std,y_train)
print("训练集准确率:",model.score(X_train_std,y_train))
print("测试集准确率: ", model.score(X_test_std, y_test))

训练集准确率: 1.0
测试集准确率:  0.8611898016997167


### lightgbm

In [51]:
import lightgbm as lgb
lgb_params = {
    'learning_rate':0.1,
    'max_bin':150,
    'num_leaves':32,    
    'max_depth':11,  
    'n_estimators':300
}
model=lgb.LGBMClassifier(**lgb_params)
model.fit(X_train_std,y_train)
print("训练集准确率:",model.score(X_train_std,y_train))
print("测试集准确率: ", model.score(X_test_std, y_test))

训练集准确率: 1.0
测试集准确率:  0.8526912181303116


### 随机森林

In [52]:
from  sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
rf = RandomForestClassifier()
#交叉验证
param = {"n_estimators":[10,100,200],"max_depth":[5,8,10]}
gc = GridSearchCV(rf,param_grid=param,cv=5)
gc.fit(X_train_std,y_train)
print("训练集准确率:",gc.score(X_train_std,y_train.astype('int')))
print("测试集准确率:",gc.score(X_test_std,y_test.astype('int')))
print("查看选择的参数模型：",gc.best_params_)
print('查看交叉验证的最优分数:',gc.best_score_)

训练集准确率: 0.9501822600243013
测试集准确率: 0.8583569405099151
查看选择的参数模型： {'max_depth': 8, 'n_estimators': 100}
查看交叉验证的最优分数: 0.8578371810449574


将上述最佳的超参数代入模型中，计算测试集的准确率

In [53]:
rf_true = RandomForestClassifier(n_estimators=200,max_depth=10,random_state=10,min_samples_split=10)
rf_true.fit(X_train_std,y_train)
predict_final = rf_true.predict(X_test_std)
prob_test = rf_true.predict_proba(X_test_std)[:,1]
print("训练集准确率:",rf_true.score(X_train_std,y_train))
print("测试集准确率: ", rf_true.score(X_test_std, y_test))
print("测试集AUC: "+str(roc_auc_score(y_test,prob_test)))
print('测试集召回率:',classification_report(y_test,predict_final))

训练集准确率: 0.9246658566221142
测试集准确率:  0.8640226628895185
测试集AUC: 0.8068617429082545
测试集召回率:               precision    recall  f1-score   support

           0       0.87      0.98      0.92       301
           1       0.64      0.17      0.27        52

   micro avg       0.86      0.86      0.86       353
   macro avg       0.76      0.58      0.60       353
weighted avg       0.84      0.86      0.83       353



## 4.预测

#### 对验证集进行同样的预处理，对离散值进行因子化，对连续值标准化

In [55]:
test.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
442,36,Non-Travel,635,Sales,10,4,Medical,1,592,2,Male,32,3,3,Sales Executive,4,Single,9980,15318,1,Y,No,14,3,4,80,0,10,3,2,10,3,9,7
1091,33,Travel_Rarely,575,Research & Development,25,3,Life Sciences,1,1545,4,Male,44,2,2,Manufacturing Director,2,Single,4320,24152,1,Y,No,13,3,4,80,0,5,2,3,5,3,0,2
981,35,Travel_Frequently,662,Sales,18,4,Marketing,1,1380,4,Female,67,3,2,Sales Executive,3,Married,4614,23288,0,Y,Yes,18,3,3,80,1,5,0,2,4,2,3,2
785,40,Travel_Rarely,1492,Research & Development,20,4,Technical Degree,1,1092,1,Male,61,3,3,Healthcare Representative,4,Married,10322,26542,4,Y,No,20,4,4,80,1,14,6,3,11,10,11,1
1332,29,Travel_Frequently,459,Research & Development,24,2,Life Sciences,1,1868,4,Male,73,2,1,Research Scientist,4,Single,2439,14753,1,Y,Yes,24,4,2,80,0,1,3,2,1,0,1,0


In [56]:
# 离散值因子化
X = test
# 离散变量
discrete = ['BusinessTravel','Department','Education', 'EducationField','Gender','JobRole','MaritalStatus', 'Over18', 'OverTime'] 
#对离散值进行one-hot encoding
BusinessTravel_dummy=pd.get_dummies(X['BusinessTravel'], prefix= 'BusinessTravel')
Department_dummy=pd.get_dummies(X['Department'], prefix= 'Department')
Education_dummy=pd.get_dummies(X['Education'], prefix= 'Education')
EducationField_dummy=pd.get_dummies(X['EducationField'], prefix= 'EducationField')
Gender_dummy=pd.get_dummies(X['Gender'], prefix= 'Gender')
JobRole_dummy=pd.get_dummies(X['JobRole'], prefix= 'JobRole')
MaritalStatus_dummy=pd.get_dummies(X['MaritalStatus'], prefix= 'MaritalStatus')
Over18_dummy=pd.get_dummies(X['Over18'], prefix= 'Over18')
OverTime_dummy=pd.get_dummies(X['OverTime'], prefix= 'OverTime')
X.drop(discrete,axis=1,inplace =True)

X=pd.concat([X,BusinessTravel_dummy,Department_dummy,Education_dummy,EducationField_dummy,Gender_dummy,
             JobRole_dummy,MaritalStatus_dummy,Over18_dummy,OverTime_dummy],axis =1)

# 连续值标准化
stdsc = StandardScaler()
#transform our training feature
X_valid_std = stdsc.fit_transform(X)
X.head()

D:\install\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\install\Anaconda\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,Age,DailyRate,DistanceFromHome,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,Education_1,Education_2,Education_3,Education_4,Education_5,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Female,Gender_Male,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
442,36,635,10,1,592,2,32,3,3,4,9980,15318,1,14,3,4,80,0,10,3,2,10,3,9,7,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0
1091,33,575,25,1,1545,4,44,2,2,2,4320,24152,1,13,3,4,80,0,5,2,3,5,3,0,2,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0
981,35,662,18,1,1380,4,67,3,2,3,4614,23288,0,18,3,3,80,1,5,0,2,4,2,3,2,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1
785,40,1492,20,1,1092,1,61,3,3,4,10322,26542,4,20,4,4,80,1,14,6,3,11,10,11,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0
1332,29,459,24,1,1868,4,73,2,1,4,2439,14753,1,24,4,2,80,0,1,3,2,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1


In [57]:
X_valid_std

array([[-0.14880903, -0.43010976,  0.0847886 , ...,  0.        ,
         0.68021068, -0.68021068],
       [-0.46815103, -0.58254453,  1.96377215, ...,  0.        ,
         0.68021068, -0.68021068],
       [-0.25525636, -0.36151411,  1.08691316, ...,  0.        ,
        -1.47013276,  1.47013276],
       ...,
       [-0.68104569, -1.41077348, -0.41627368, ...,  0.        ,
        -1.47013276,  1.47013276],
       [ 0.06408564,  1.16029307, -0.91733596, ...,  0.        ,
         0.68021068, -0.68021068],
       [ 0.27698031, -1.10590393, -0.16574254, ...,  0.        ,
        -1.47013276,  1.47013276]])

In [69]:
prob_valid = lg.predict_proba(X_valid_std)[:,1]

In [70]:
prob_valid

array([9.53220130e-02, 1.61268591e-01, 6.69543345e-01, 1.18500519e-01,
       7.52885108e-01, 5.17442949e-02, 3.59713708e-01, 3.34120836e-02,
       4.63352523e-04, 2.11392738e-01, 6.36264369e-02, 4.28006007e-01,
       1.52813306e-02, 3.47814557e-01, 5.58643253e-02, 7.43079299e-03,
       1.08187097e-01, 1.97281934e-02, 1.69635874e-02, 2.75984559e-01,
       1.28245262e-01, 1.90828044e-02, 1.93733501e-02, 5.12852955e-02,
       7.89591667e-01, 5.10876416e-01, 7.95302460e-02, 3.15185983e-02,
       7.31840285e-01, 1.11012401e-01, 3.62812442e-03, 2.16587719e-01,
       3.85934667e-02, 4.22300016e-02, 4.22690268e-02, 6.87688194e-03,
       1.40370274e-01, 6.21056961e-02, 4.86373392e-02, 9.49441768e-02,
       2.28558163e-02, 1.84249776e-02, 3.30208295e-04, 3.25470828e-03,
       1.15890021e-02, 4.53812395e-01, 3.77648880e-01, 3.48752927e-04,
       2.98398239e-01, 3.85215526e-01, 3.56390976e-02, 8.91207970e-01,
       2.23886499e-02, 2.90985341e-01, 2.48284289e-01, 8.67481397e-02,
      

In [71]:
result = pd.DataFrame({'Useid':X.index, 'Attrition':prob_valid.astype(np.int32)})
result

,Useid,Attrition
0,442,0
1,1091,0
2,981,0
3,785,0
4,1332,0
5,501,0
6,1058,0
7,1253,0
8,751,0
9,122,0


In [72]:
result['Attrition'].value_counts() 

0    294
Name: Attrition, dtype: int64